# EX8-STREAM: Spark Structured Streaming

Your assignment: complete the `TODO`'s and include also the **output of each cell**.

#### You may need to read the [Structured Streaming API Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/index.html) to complete this lab.

### Step 1: **[PLAN A]** Start Spark Session

In [ ]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

    # cluster mode
spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("spark://spark:7077") \
	    	.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
            .config("spark.hadoop.fs.s3a.access.key", "pdm_minio") \
            .config("spark.hadoop.fs.s3a.secret.key", "pdm_minio") \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
	    	.getOrCreate()

### Step 1: **[PLAN B]** Start Spark Session

In [1]:
from pyspark.sql import SparkSession
Start
try:
    spark.stop()
except NameError:
    print("SparkContext not defined")
    

# local mode
spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("local[*]") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

SparkContext not defined


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/13 00:34:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2: Static Dataframe of words

In [2]:
#words_df = spark.read.csv("s3a://public/100words.txt.gz") # plan A
words_df = spark.read.csv("data/100words.txt.gz") # plan B
words_df = words_df.withColumnRenamed("_c0", "word")
words_df.show()

+----------+
|      word|
+----------+
|     apple|
|     sunny|
| bookshelf|
|  computer|
|     pizza|
|    friend|
|     happy|
|    flower|
|basketball|
|       dog|
|     beach|
|  vacation|
|     piano|
|  elephant|
|    coffee|
|  sunshine|
| butterfly|
|  mountain|
|    guitar|
|       cat|
+----------+
only showing top 20 rows



### Step 3: Get meaning for each word (use [Free Dictionary API](https://dictionaryapi.dev/))

In [3]:
from pyspark.sql.functions import *
import requests_ratelimiter

def get_word_meaning(word, session):
    url = f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}"
    response = session.get(url)
    response.raise_for_status()  # Ensure the request was successful
    json_data = response.json()

    try:
        meaning = json_data[0]['meanings'][0]['definitions'][0]['definition']
    except:
        meaning = "__NOT_FOUND__"

    return meaning


try:
    words_with_meaning_df.cache()
    words_with_meaning_df.show()
except NameError:
    print("words_with_meaning_df not defined")
    meanings = []
    session = requests_ratelimiter.LimiterSession(per_second=1)
    for word in [r.word for r in words_df.collect()]:
        meanings.append((word, get_word_meaning(word, session)))
        print(word)
    words_with_meaning_df = spark.createDataFrame(meanings, ["word", "meaning"])
    words_with_meaning_df.cache()
    words_with_meaning_df.show()

words_with_meaning_df not defined
apple
sunny
bookshelf
computer
pizza
friend
happy
flower
basketball
dog
beach
vacation
piano
elephant
coffee
sunshine
butterfly
mountain
guitar
cat
phone
birthday
gift
school
homework
music
swimming
library
science
fiction
poem
tree
bird
water
city
park
running
jumping
cycling
sky
cloud
starfish
ocean
wave
shell
pearl
snail
speed
racing
winner
treasure
pirate
shipwreck
mermaid
fish
coral
reef
surfboard
kayak
paddle
canoe
tent
camping
marshmallow
s'mores
campfire
sparkler
firework
dragonfly
monarch
milkshake
ice cream
cookie
brownie
sundae
peanut butter
jelly
grape
strawberry
blueberry
raspberry
kiwi
pineapple
orange
banana
avocado
lemon
lime
mint
basil
rosemary
thyme
oregano
cinnamon
nutmeg
gingerbread
hot chocolate
marshmallow
stuffing
turkey


+----------+--------------------+
|      word|             meaning|
+----------+--------------------+
|     apple|A common, round f...|
|     sunny|          A sunfish.|
| bookshelf|A shelf or shelve...|
|  computer|A person employed...|
|     pizza|A baked Italian d...|
|    friend|A person other th...|
|     happy|A happy event, th...|
|    flower|A colorful, consp...|
|basketball|A sport in which ...|
|       dog|A mammal, Canis f...|
|     beach|The shore of a bo...|
|  vacation|Freedom from some...|
|     piano|A percussive keyb...|
|  elephant|A mammal of the o...|
|    coffee|A beverage made b...|
|  sunshine|The direct rays, ...|
| butterfly|A flying insect o...|
|  mountain|An elevation of l...|
|    guitar|A stringed musica...|
|       cat|An animal of the ...|
+----------+--------------------+
only showing top 20 rows



### Step 4: **[PLAN A]** Create a stream of sentences using existing socket stream (LAB)

In [ ]:
words_stream = spark \
    .readStream.format("socket") \
    .option("host", "socketstreamserver") \
    .option("port", 12345) \
    .load()

### Step 4: **[PLAN B]** Create a socket stream and create a stream of sentences from that (NOTEBOOK LOCAL)

1. Before running the cell below, start socket stream from existing script `hostdir/bin/cmd.sh` using a notebook terminal.
2. Make sure it is running properly.
3. Create a spark stream using the command below

In [4]:
words_stream = spark \
    .readStream.format("socket") \
    .option("host", "localhost") \
    .option("port", 12345) \
    .load()

25/05/13 00:38:10 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


### Step 5: Start stream just to visualize some of its values (for 10 seconds)

In [9]:
words_stream_writer = words_stream.writeStream.format("console").outputMode("append")
words_stream_writer = words_stream_writer.trigger(processingTime="1 second")
words_stream_query = words_stream_writer.start()

try:
    words_stream_query.awaitTermination(10)
except Exception as e:
    print(f"Streaming failed: {str(e)}")
finally:
    words_stream_query.stop()

25/05/13 00:57:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-82f7573c-48c6-497e-9936-1f7f4a70cf89. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/13 00:57:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|12 fish sky orang...|
+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+
|   value|
+--------+
|9 grape |
+--------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|         17 avocado |
|18 dog lemon elep...|
+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|3 happy jelly dra...|
+--------------------+

-------------------------------------------
Batch: 5
--

### Step 6: Transform the stream as requested `#TODO`

1. Each line of the stream starts with a number, let us call this number `user_id`. The rest of the line comprises a set of words generated by this user.
2. For each user request you must take the corresponding words, get the meaning of each word (static dataframe) and return the responses as a new stream of `user_id, [<meaning of word 1>, <meaning of word 2>, ... ]`
3. Let the stream running on console for 10 seconds.

In [16]:
from pyspark.sql.functions import split, col, explode, array, collect_list

processed_stream = words_stream \
    .select(
        split(col("value"), " ", 2).alias("data")
    ) \
    .select(
        col("data").getItem(0).alias("user_id"),
        split(col("data").getItem(1), " ").alias("words")
    ) \
    .withColumn("word", explode(col("words")))

enriched_stream = processed_stream \
    .join(words_with_meaning_df, "word", "left") \
    .groupBy("user_id") \
    .agg(collect_list("meaning").alias("meanings"))

query = enriched_stream \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime="1 second") \
    .option("truncate", "false") \
    .start()

try:
    query.awaitTermination(10)
except Exception as e:
    print(f"Streaming failed: {str(e)}")
finally:
    query.stop()

25/05/13 01:10:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fb5a4447-7bf0-4857-bfe0-af57c41ed4f1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/13 01:10:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|user_id|meanings|
+-------+--------+
+-------+--------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|meanings                                                                                                                                                                                                                                                                                                                                   

25/05/13 01:10:29 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 7, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/05/13 01:10:29 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 7, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/05/13 01:10:29 WARN Shell: Interrupted while joining on: Thread[Thread-102755,5,main]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1042)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1002)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.ha

### Step 7: Transform the stream as requested `#TODO`

1. Again, from the stream of lines `words_stream`
2. Map each line to rows of `word,user_id` (hint: use `explode` and `split`)
3. From this new stream, group by word and aggregate the set of user IDs that asked for that specific word.
4. Generate a stream of `<list of user IDs> <word> <meaning of word>`
5. Let the resulting stream running for 20 seconds.

In [14]:
from pyspark.sql.functions import split, col, explode, collect_set, array_join

word_user_stream = words_stream \
    .select(
        split(col("value"), " ", 2).alias("data")
    ) \
    .select(
        col("data").getItem(0).alias("user_id"),
        split(col("data").getItem(1), " ").alias("words")
    ) \
    .select(
        col("user_id"),
        explode(col("words")).alias("word")
    )

result_stream = word_user_stream \
    .join(words_with_meaning_df, "word", "left") \
    .groupBy("word", "meaning") \
    .agg(
        collect_set("user_id").alias("user_ids"),
        count("*").alias("request_count")
    ) \
    .select(
        array_join(col("user_ids"), ",").alias("user_ids"),
        col("word"),
        col("meaning"),
        col("request_count")
    )

query = result_stream \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .option("numRows", 20) \
    .option("checkpointLocation", "file:///tmp/word_user_checkpoint") \
    .trigger(processingTime="1 second") \
    .start()

try:
    query.awaitTermination(20)
except KeyboardInterrupt:
    pass
finally:
    query.stop()

25/05/13 01:08:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+----+-------+-------------+
|user_ids|word|meaning|request_count|
+--------+----+-------+-------------+
+--------+----+-------+-------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|user_ids|word     |meaning                                                                                                                                                      |request_count|
+--------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|7       |brownie  |A small square piece of rich cake, usually made with c

-------------------------------------------
Batch: 13
-------------------------------------------
+--------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|user_ids|word    |meaning                                                                                                                                                                                                                    |request_count|
+--------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|4,7     |brownie |A small square piece of rich cake, usually made with chocolate.                                                          